In [2]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.0.0


In [20]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)   #[60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)   #[10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)  #[60000]
        self.test_label = self.test_label.astype(np.int32)   #[10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, np.shape(self.train_data)[0], size=batch_size)
        return self.train_data[index, :], self.train_label[index]

class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units=200, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10, activation=tf.nn.softmax)
  
    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dense1(x)
        output = self.dense2(x)
        return output

num_epochs = 20
batch_size = 200
learning_rate = 0.001

def train():
    model = MLP()
    data_loader = MNISTLoader()
    
    summary_writer = tf.summary.create_file_writer('./tensorboard')     # 训练过程可视化，参数为记录文件所保存的目录
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    checkpoint = tf.train.Checkpoint(myAwesomeModel=model)
    # 使用tf.train.CheckpointManager管理Checkpoint
    manager = tf.train.CheckpointManager(checkpoint, directory='./save', max_to_keep=3)
    sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()  #正确率评估
    
    num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
    for batch_index in range(num_batches):
        X, y = data_loader.get_batch(batch_size)
        with tf.GradientTape() as tape:
            y_pred = model(X)
            loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
            loss = tf.reduce_mean(loss)
            sparse_categorical_accuracy.update_state(y_true=y, y_pred=y_pred)
            print("batch %d: loss %f accuracy: %f" % (batch_index, loss.numpy(), sparse_categorical_accuracy.result()))
        grads = tape.gradient(loss, model.variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
        
        
        with summary_writer.as_default():                               # 希望使用的记录器
            tf.summary.scalar("loss", loss, step=batch_index)
            tf.summary.scalar("accuracy", sparse_categorical_accuracy.result(), step=batch_index)
            #tf.summary.scalar("MyScalar", my_scalar, step=batch_index)  # 还可以添加其他自定义的变量
        
        if batch_index % 100 == 0:
            # 使用CheckpointManager保存模型参数到文件并自定义编号
            path = manager.save(checkpoint_number=batch_index)         
            print("model saved to %s" % path)

def test():
    model_to_be_restored = MLP()
    data_loader = MNISTLoader()
    checkpoint = tf.train.Checkpoint(myAwesomeModel=model_to_be_restored)      
    checkpoint.restore(tf.train.latest_checkpoint('./save'))
    sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    num_batches = int(data_loader.num_test_data // batch_size)
    for batch_index in range(num_batches):
        start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
        y_pred = model_to_be_restored.predict(data_loader.test_data[start_index: end_index])
        sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
    print("test accuracy: %f" % sparse_categorical_accuracy.result()) 

当我们要对训练过程可视化时，在代码目录打开终端（如需要的话进入 TensorFlow 的 conda 环境），运行:

tensorboard --logdir=./tensorboard

然后使用浏览器访问命令行程序所输出的网址（一般是 http:// 计算机名称：6006），即可访问 TensorBoard 的可视界面

如果需要重新训练，需要删除掉记录文件夹内的信息并重启 TensorBoard（或者建立一个新的记录文件夹并开启 TensorBoard， --logdir 参数设置为新建立的文件夹）；

记录文件夹目录保持全英文。

In [22]:
if __name__ == '__main__':
    train()
    #test()

batch 0: loss 2.425844 accuracy: 0.055000
model saved to ./save\ckpt-0
batch 1: loss 2.225153 accuracy: 0.130000
batch 2: loss 1.994172 accuracy: 0.215000
batch 3: loss 1.877686 accuracy: 0.283750
batch 4: loss 1.778287 accuracy: 0.345000
batch 5: loss 1.588717 accuracy: 0.405833
batch 6: loss 1.555903 accuracy: 0.451429
batch 7: loss 1.393481 accuracy: 0.484375
batch 8: loss 1.335056 accuracy: 0.512222
batch 9: loss 1.300263 accuracy: 0.532000
batch 10: loss 1.129956 accuracy: 0.553636
batch 11: loss 1.116520 accuracy: 0.570417
batch 12: loss 1.093734 accuracy: 0.583077
batch 13: loss 0.995653 accuracy: 0.596786
batch 14: loss 1.018828 accuracy: 0.608667
batch 15: loss 0.929597 accuracy: 0.619687
batch 16: loss 0.862407 accuracy: 0.628529
batch 17: loss 0.872835 accuracy: 0.638056
batch 18: loss 0.734545 accuracy: 0.649211
batch 19: loss 0.674300 accuracy: 0.660250
batch 20: loss 0.674223 accuracy: 0.669524
batch 21: loss 0.627961 accuracy: 0.677500
batch 22: loss 0.581243 accuracy: 0

model saved to ./save\ckpt-200
batch 201: loss 0.232207 accuracy: 0.886139
batch 202: loss 0.146546 accuracy: 0.886552
batch 203: loss 0.178664 accuracy: 0.886887
batch 204: loss 0.269100 accuracy: 0.887146
batch 205: loss 0.205735 accuracy: 0.887427
batch 206: loss 0.199788 accuracy: 0.887681
batch 207: loss 0.303717 accuracy: 0.887812
batch 208: loss 0.205603 accuracy: 0.888110
batch 209: loss 0.254111 accuracy: 0.888357
batch 210: loss 0.221384 accuracy: 0.888602
batch 211: loss 0.193473 accuracy: 0.888868
batch 212: loss 0.256235 accuracy: 0.889061
batch 213: loss 0.185653 accuracy: 0.889346
batch 214: loss 0.159521 accuracy: 0.889721
batch 215: loss 0.172696 accuracy: 0.890000
batch 216: loss 0.186394 accuracy: 0.890276
batch 217: loss 0.197353 accuracy: 0.890505
batch 218: loss 0.124629 accuracy: 0.890868
batch 219: loss 0.261852 accuracy: 0.891000
batch 220: loss 0.180052 accuracy: 0.891267
batch 221: loss 0.299382 accuracy: 0.891306
batch 222: loss 0.283187 accuracy: 0.891502
b

batch 398: loss 0.242758 accuracy: 0.915689
batch 399: loss 0.142628 accuracy: 0.915775
batch 400: loss 0.161036 accuracy: 0.915835
model saved to ./save\ckpt-400
batch 401: loss 0.127148 accuracy: 0.915958
batch 402: loss 0.213275 accuracy: 0.916042
batch 403: loss 0.072643 accuracy: 0.916213
batch 404: loss 0.134382 accuracy: 0.916321
batch 405: loss 0.163977 accuracy: 0.916367
batch 406: loss 0.109140 accuracy: 0.916523
batch 407: loss 0.184764 accuracy: 0.916581
batch 408: loss 0.255500 accuracy: 0.916626
batch 409: loss 0.213414 accuracy: 0.916695
batch 410: loss 0.193755 accuracy: 0.916764
batch 411: loss 0.234134 accuracy: 0.916796
batch 412: loss 0.165643 accuracy: 0.916913
batch 413: loss 0.132165 accuracy: 0.917053
batch 414: loss 0.133691 accuracy: 0.917157
batch 415: loss 0.096855 accuracy: 0.917308
batch 416: loss 0.183356 accuracy: 0.917398
batch 417: loss 0.102575 accuracy: 0.917548
batch 418: loss 0.273568 accuracy: 0.917578
batch 419: loss 0.174450 accuracy: 0.917690
b

batch 591: loss 0.127288 accuracy: 0.929029
batch 592: loss 0.137808 accuracy: 0.929089
batch 593: loss 0.231446 accuracy: 0.929066
batch 594: loss 0.116334 accuracy: 0.929118
batch 595: loss 0.124516 accuracy: 0.929178
batch 596: loss 0.106653 accuracy: 0.929238
batch 597: loss 0.155653 accuracy: 0.929273
batch 598: loss 0.097893 accuracy: 0.929357
batch 599: loss 0.187984 accuracy: 0.929400
batch 600: loss 0.147797 accuracy: 0.929443
model saved to ./save\ckpt-600
batch 601: loss 0.163069 accuracy: 0.929485
batch 602: loss 0.185491 accuracy: 0.929502
batch 603: loss 0.129574 accuracy: 0.929545
batch 604: loss 0.153001 accuracy: 0.929595
batch 605: loss 0.193126 accuracy: 0.929629
batch 606: loss 0.092847 accuracy: 0.929703
batch 607: loss 0.231845 accuracy: 0.929712
batch 608: loss 0.109923 accuracy: 0.929762
batch 609: loss 0.169788 accuracy: 0.929820
batch 610: loss 0.135825 accuracy: 0.929861
batch 611: loss 0.111354 accuracy: 0.929926
batch 612: loss 0.078509 accuracy: 0.930008
b

batch 785: loss 0.117611 accuracy: 0.938384
batch 786: loss 0.121138 accuracy: 0.938443
batch 787: loss 0.112191 accuracy: 0.938464
batch 788: loss 0.134518 accuracy: 0.938498
batch 789: loss 0.133463 accuracy: 0.938525
batch 790: loss 0.092494 accuracy: 0.938571
batch 791: loss 0.129171 accuracy: 0.938599
batch 792: loss 0.126356 accuracy: 0.938632
batch 793: loss 0.079046 accuracy: 0.938671
batch 794: loss 0.068989 accuracy: 0.938730
batch 795: loss 0.063515 accuracy: 0.938800
batch 796: loss 0.081106 accuracy: 0.938852
batch 797: loss 0.091618 accuracy: 0.938897
batch 798: loss 0.157626 accuracy: 0.938917
batch 799: loss 0.110318 accuracy: 0.938950
batch 800: loss 0.073858 accuracy: 0.938995
model saved to ./save\ckpt-800
batch 801: loss 0.086016 accuracy: 0.939046
batch 802: loss 0.074149 accuracy: 0.939091
batch 803: loss 0.087893 accuracy: 0.939136
batch 804: loss 0.132895 accuracy: 0.939155
batch 805: loss 0.117861 accuracy: 0.939175
batch 806: loss 0.061822 accuracy: 0.939238
b

...

batch 5144: loss 0.006863 accuracy: 0.982869
batch 5145: loss 0.005075 accuracy: 0.982872
batch 5146: loss 0.010018 accuracy: 0.982875
batch 5147: loss 0.010983 accuracy: 0.982878
batch 5148: loss 0.004415 accuracy: 0.982881
batch 5149: loss 0.009131 accuracy: 0.982884
batch 5150: loss 0.008141 accuracy: 0.982887
batch 5151: loss 0.007938 accuracy: 0.982889
batch 5152: loss 0.007666 accuracy: 0.982893
batch 5153: loss 0.010496 accuracy: 0.982896
batch 5154: loss 0.010298 accuracy: 0.982899
batch 5155: loss 0.004927 accuracy: 0.982902
batch 5156: loss 0.003895 accuracy: 0.982906
batch 5157: loss 0.006890 accuracy: 0.982908
batch 5158: loss 0.006827 accuracy: 0.982911
batch 5159: loss 0.008030 accuracy: 0.982915
batch 5160: loss 0.009254 accuracy: 0.982918
batch 5161: loss 0.007497 accuracy: 0.982921
batch 5162: loss 0.002639 accuracy: 0.982925
batch 5163: loss 0.008237 accuracy: 0.982928
batch 5164: loss 0.014015 accuracy: 0.982930
batch 5165: loss 0.006182 accuracy: 0.982934
batch 5166

batch 5327: loss 0.006853 accuracy: 0.983407
batch 5328: loss 0.007569 accuracy: 0.983410
batch 5329: loss 0.008454 accuracy: 0.983412
batch 5330: loss 0.003462 accuracy: 0.983415
batch 5331: loss 0.002968 accuracy: 0.983418
batch 5332: loss 0.002910 accuracy: 0.983421
batch 5333: loss 0.006240 accuracy: 0.983424
batch 5334: loss 0.014603 accuracy: 0.983426
batch 5335: loss 0.003323 accuracy: 0.983430
batch 5336: loss 0.004619 accuracy: 0.983433
batch 5337: loss 0.007926 accuracy: 0.983436
batch 5338: loss 0.011607 accuracy: 0.983438
batch 5339: loss 0.011986 accuracy: 0.983441
batch 5340: loss 0.006551 accuracy: 0.983444
batch 5341: loss 0.004817 accuracy: 0.983447
batch 5342: loss 0.005436 accuracy: 0.983450
batch 5343: loss 0.007859 accuracy: 0.983453
batch 5344: loss 0.001517 accuracy: 0.983456
batch 5345: loss 0.004891 accuracy: 0.983460
batch 5346: loss 0.004194 accuracy: 0.983463
batch 5347: loss 0.010658 accuracy: 0.983465
batch 5348: loss 0.004316 accuracy: 0.983468
batch 5349

batch 5515: loss 0.006083 accuracy: 0.983939
batch 5516: loss 0.003026 accuracy: 0.983941
batch 5517: loss 0.005096 accuracy: 0.983944
batch 5518: loss 0.003250 accuracy: 0.983947
batch 5519: loss 0.003782 accuracy: 0.983950
batch 5520: loss 0.004804 accuracy: 0.983953
batch 5521: loss 0.004948 accuracy: 0.983956
batch 5522: loss 0.002322 accuracy: 0.983959
batch 5523: loss 0.005494 accuracy: 0.983962
batch 5524: loss 0.004000 accuracy: 0.983965
batch 5525: loss 0.003411 accuracy: 0.983968
batch 5526: loss 0.005552 accuracy: 0.983971
batch 5527: loss 0.009796 accuracy: 0.983973
batch 5528: loss 0.006337 accuracy: 0.983976
batch 5529: loss 0.003340 accuracy: 0.983979
batch 5530: loss 0.005606 accuracy: 0.983982
batch 5531: loss 0.003412 accuracy: 0.983985
batch 5532: loss 0.012770 accuracy: 0.983987
batch 5533: loss 0.012004 accuracy: 0.983990
batch 5534: loss 0.004838 accuracy: 0.983993
batch 5535: loss 0.004359 accuracy: 0.983996
batch 5536: loss 0.004156 accuracy: 0.983999
batch 5537

batch 5697: loss 0.009669 accuracy: 0.984415
batch 5698: loss 0.008845 accuracy: 0.984417
batch 5699: loss 0.002927 accuracy: 0.984420
batch 5700: loss 0.010463 accuracy: 0.984422
model saved to ./save\ckpt-5700
batch 5701: loss 0.006136 accuracy: 0.984425
batch 5702: loss 0.007046 accuracy: 0.984428
batch 5703: loss 0.015252 accuracy: 0.984430
batch 5704: loss 0.004222 accuracy: 0.984433
batch 5705: loss 0.001816 accuracy: 0.984436
batch 5706: loss 0.004467 accuracy: 0.984438
batch 5707: loss 0.002949 accuracy: 0.984441
batch 5708: loss 0.029153 accuracy: 0.984443
batch 5709: loss 0.005642 accuracy: 0.984446
batch 5710: loss 0.003275 accuracy: 0.984448
batch 5711: loss 0.004992 accuracy: 0.984451
batch 5712: loss 0.004332 accuracy: 0.984454
batch 5713: loss 0.003835 accuracy: 0.984457
batch 5714: loss 0.006291 accuracy: 0.984459
batch 5715: loss 0.006479 accuracy: 0.984462
batch 5716: loss 0.002942 accuracy: 0.984465
batch 5717: loss 0.004671 accuracy: 0.984467
batch 5718: loss 0.0038

batch 5885: loss 0.011333 accuracy: 0.984885
batch 5886: loss 0.012039 accuracy: 0.984887
batch 5887: loss 0.002435 accuracy: 0.984890
batch 5888: loss 0.010515 accuracy: 0.984891
batch 5889: loss 0.004425 accuracy: 0.984894
batch 5890: loss 0.003007 accuracy: 0.984896
batch 5891: loss 0.006264 accuracy: 0.984899
batch 5892: loss 0.004073 accuracy: 0.984902
batch 5893: loss 0.007077 accuracy: 0.984903
batch 5894: loss 0.008696 accuracy: 0.984905
batch 5895: loss 0.007659 accuracy: 0.984907
batch 5896: loss 0.001705 accuracy: 0.984909
batch 5897: loss 0.007120 accuracy: 0.984912
batch 5898: loss 0.006315 accuracy: 0.984914
batch 5899: loss 0.006484 accuracy: 0.984917
batch 5900: loss 0.002106 accuracy: 0.984919
model saved to ./save\ckpt-5900
batch 5901: loss 0.011074 accuracy: 0.984921
batch 5902: loss 0.005378 accuracy: 0.984924
batch 5903: loss 0.017370 accuracy: 0.984925
batch 5904: loss 0.007893 accuracy: 0.984927
batch 5905: loss 0.008117 accuracy: 0.984930
batch 5906: loss 0.0084

In [23]:
    test()

test accuracy: 0.980800


In [ ]:
#mnist 极简
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, verbose=1)
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:{}, acc:{}'.format(test_loss, test_acc))